# Variable Reflection Experiment Runner

This notebook allows you to run experiments with configurable parameters:
- **Benchmarks**: ETHICS, MoralChoice, MORABLES
- **Reflection Levels**: 0-5
- **Thinking Conditions**: ON/OFF
- **Sample Size**: Number of items per benchmark
- **Number of Runs**: For consistency analysis

## 1. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from pathlib import Path
from tqdm.notebook import tqdm
from datetime import datetime
from IPython.display import display, Markdown

# Load environment variables
with open('.env', 'r') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#'):
            match = re.match(r'(\w+)\s*=\s*["\']?([^"\']+)["\']?', line)
            if match:
                os.environ[match.group(1)] = match.group(2)

# Import project modules
from prompts import get_ethics_prompt, get_moralchoice_prompt, get_morables_prompt
from src.api import call_with_rate_limit
from src.extraction import (
    extract_ethics_answer,
    extract_moralchoice_with_confidence,
    extract_morables_answer,
    extract_confidence_score,
    categorize_confidence,
    count_reasoning_markers,
    count_uncertainty_markers
)
import config

print(f"Model: {config.MODEL}")
print(f"API configured: {'ANTHROPIC_API_KEY' in os.environ}")

## 2. Experiment Configuration

**Modify these parameters to customize your experiment:**

In [ ]:
# =============================================================================
# EXPERIMENT PARAMETERS - MODIFY THESE
# =============================================================================

# Which benchmarks to run (set to True/False)
RUN_ETHICS = True
RUN_MORALCHOICE = True
RUN_MORABLES = True

# Reflection levels to test (0-5)
# Level 0: Direct, Level 1: Minimal, Level 2: CoT, Level 3: Structured
# Level 4: Adversarial, Level 5: Two-Pass (2 API calls per item)
LEVELS = [0, 1, 2, 3, 4, 5]

# Thinking conditions
THINKING_CONDITIONS = [False, True]  # [OFF, ON]

# Sample size per benchmark (set to None for full dataset)
SAMPLE_SIZE = 10  # Start small for testing, increase for full experiment

# Number of runs (for consistency analysis)
N_RUNS = 1

# Whether to include confidence scoring
INCLUDE_CONFIDENCE = True

# Output directory
OUTPUT_DIR = "results/notebook_runs"

# =============================================================================
# Calculate estimated API calls
# =============================================================================
def estimate_api_calls():
    calls_per_item = len(LEVELS) * len(THINKING_CONDITIONS) * N_RUNS
    # Level 5 requires 2 calls per item, others require 1
    level_5_extra = (1 if 5 in LEVELS else 0) * len(THINKING_CONDITIONS) * N_RUNS
    
    total = 0
    if RUN_ETHICS:
        total += SAMPLE_SIZE * (calls_per_item + level_5_extra)
    if RUN_MORALCHOICE:
        total += SAMPLE_SIZE * (calls_per_item + level_5_extra)
    if RUN_MORABLES:
        total += SAMPLE_SIZE * (calls_per_item + level_5_extra)
    return total

print("=" * 50)
print("EXPERIMENT CONFIGURATION")
print("=" * 50)
print(f"Benchmarks: {', '.join([b for b, run in [('ETHICS', RUN_ETHICS), ('MoralChoice', RUN_MORALCHOICE), ('MORABLES', RUN_MORABLES)] if run])}")
print(f"Levels: {LEVELS}")
print(f"Thinking conditions: {['OFF' if not t else 'ON' for t in THINKING_CONDITIONS]}")
print(f"Sample size: {SAMPLE_SIZE}")
print(f"Runs: {N_RUNS}")
print(f"Include confidence: {INCLUDE_CONFIDENCE}")
print(f"\nEstimated API calls: ~{estimate_api_calls()}")

## 3. Load Data

In [ ]:
# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Load datasets
datasets = {}

if RUN_ETHICS:
    ethics_df = pd.read_csv("data/ethics_sample.csv")
    if SAMPLE_SIZE and len(ethics_df) > SAMPLE_SIZE:
        # Balance by subscale
        per_subscale = SAMPLE_SIZE // 3
        subscales = []
        for subscale in ['commonsense', 'deontology', 'virtue']:
            subset = ethics_df[ethics_df['subscale'] == subscale].head(per_subscale)
            subscales.append(subset)
        ethics_df = pd.concat(subscales, ignore_index=True)
    datasets['ethics'] = ethics_df
    print(f"ETHICS: {len(ethics_df)} items")
    print(f"  Subscales: {ethics_df['subscale'].value_counts().to_dict()}")

if RUN_MORALCHOICE:
    mc_df = pd.read_csv("data/moralchoice_sample.csv")
    if SAMPLE_SIZE and len(mc_df) > SAMPLE_SIZE:
        # Balance by ambiguity
        per_level = SAMPLE_SIZE // 2
        low_amb = mc_df[mc_df['ambiguity'] == 'low'].head(per_level)
        high_amb = mc_df[mc_df['ambiguity'] == 'high'].head(per_level)
        mc_df = pd.concat([low_amb, high_amb], ignore_index=True)
    datasets['moralchoice'] = mc_df
    print(f"\nMoralChoice: {len(mc_df)} items")
    if 'ambiguity' in mc_df.columns:
        print(f"  Ambiguity: {mc_df['ambiguity'].value_counts().to_dict()}")

if RUN_MORABLES:
    morables_path = "data/morables/morables_sample.csv"
    if Path(morables_path).exists():
        morables_df = pd.read_csv(morables_path)
        if SAMPLE_SIZE and len(morables_df) > SAMPLE_SIZE:
            morables_df = morables_df.sample(n=SAMPLE_SIZE, random_state=config.RANDOM_SEED)
        datasets['morables'] = morables_df
        print(f"\nMORABLES: {len(morables_df)} items")
    else:
        print(f"\nMORABLES: Not found at {morables_path}")
        print("  Run 'python prepare_data.py' to download the dataset")
        RUN_MORABLES = False

## 4. Experiment Runner Functions

In [ ]:
def run_single_ethics(row, level, thinking, include_confidence=True):
    """Run single ETHICS item."""
    if level == 5:
        prompt1 = get_ethics_prompt(5, row['scenario'], include_confidence=include_confidence)
        response1 = call_with_rate_limit(prompt1, thinking)
        
        prompt2 = get_ethics_prompt(5, row['scenario'], response1.content, include_confidence=include_confidence)
        response2 = call_with_rate_limit(prompt2, thinking)
        
        return {
            'content': response2.content,
            'thinking': response2.thinking,
            'full_response': f"[PASS1]\n{response1.content}\n\n[PASS2]\n{response2.content}",
            'input_tokens': response1.input_tokens + response2.input_tokens,
            'output_tokens': response1.output_tokens + response2.output_tokens,
        }
    else:
        prompt = get_ethics_prompt(level, row['scenario'], include_confidence=include_confidence)
        response = call_with_rate_limit(prompt, thinking)
        return {
            'content': response.content,
            'thinking': response.thinking,
            'full_response': response.content,
            'input_tokens': response.input_tokens,
            'output_tokens': response.output_tokens,
        }


def run_single_moralchoice(row, level, thinking, include_confidence=True):
    """Run single MoralChoice item."""
    context = row.get('context', '')
    
    if level == 5:
        prompt1 = get_moralchoice_prompt(5, context, row['option_a'], row['option_b'],
                                         include_confidence=include_confidence)
        response1 = call_with_rate_limit(prompt1, thinking)
        
        prompt2 = get_moralchoice_prompt(5, context, row['option_a'], row['option_b'],
                                         previous_response=response1.content,
                                         include_confidence=include_confidence)
        response2 = call_with_rate_limit(prompt2, thinking)
        
        return {
            'content': response2.content,
            'thinking': response2.thinking,
            'full_response': f"[PASS1]\n{response1.content}\n\n[PASS2]\n{response2.content}",
            'input_tokens': response1.input_tokens + response2.input_tokens,
            'output_tokens': response1.output_tokens + response2.output_tokens,
        }
    else:
        prompt = get_moralchoice_prompt(level, context, row['option_a'], row['option_b'],
                                        include_confidence=include_confidence)
        response = call_with_rate_limit(prompt, thinking)
        return {
            'content': response.content,
            'thinking': response.thinking,
            'full_response': response.content,
            'input_tokens': response.input_tokens,
            'output_tokens': response.output_tokens,
        }


def run_single_morables(row, level, thinking, include_confidence=True):
    """Run single MORABLES item."""
    options = [row['option_a'], row['option_b'], row['option_c'], row['option_d'], row['option_e']]
    
    if level == 5:
        prompt1 = get_morables_prompt(5, row['fable'], options, include_confidence=include_confidence)
        response1 = call_with_rate_limit(prompt1, thinking)
        
        prompt2 = get_morables_prompt(5, row['fable'], options,
                                       previous_response=response1.content,
                                       include_confidence=include_confidence)
        response2 = call_with_rate_limit(prompt2, thinking)
        
        return {
            'content': response2.content,
            'thinking': response2.thinking,
            'full_response': f"[PASS1]\n{response1.content}\n\n[PASS2]\n{response2.content}",
            'input_tokens': response1.input_tokens + response2.input_tokens,
            'output_tokens': response1.output_tokens + response2.output_tokens,
        }
    else:
        prompt = get_morables_prompt(level, row['fable'], options, include_confidence=include_confidence)
        response = call_with_rate_limit(prompt, thinking)
        return {
            'content': response.content,
            'thinking': response.thinking,
            'full_response': response.content,
            'input_tokens': response.input_tokens,
            'output_tokens': response.output_tokens,
        }

print("Runner functions defined.")

## 5. Run ETHICS Experiment

In [ ]:
if RUN_ETHICS:
    print("=" * 60)
    print("RUNNING ETHICS EXPERIMENT")
    print("=" * 60)
    
    ethics_results = []
    ethics_df = datasets['ethics']
    
    total_conditions = len(LEVELS) * len(THINKING_CONDITIONS) * N_RUNS
    condition_num = 0
    
    for run in range(N_RUNS):
        for thinking in THINKING_CONDITIONS:
            for level in LEVELS:
                condition_num += 1
                thinking_label = "ON" if thinking else "OFF"
                
                print(f"\n[{condition_num}/{total_conditions}] Run {run+1}, Level {level}, Thinking {thinking_label}")
                
                for _, row in tqdm(ethics_df.iterrows(), total=len(ethics_df), 
                                   desc=f"L{level}-{thinking_label}"):
                    try:
                        response_data = run_single_ethics(row, level, thinking, INCLUDE_CONFIDENCE)
                        
                        extracted = extract_ethics_answer(response_data['content'])
                        confidence = extract_confidence_score(response_data['content']) if INCLUDE_CONFIDENCE else None
                        
                        ethics_results.append({
                            'item_id': row['item_id'],
                            'subscale': row['subscale'],
                            'scenario': row['scenario'][:200],
                            'correct_answer': row['label'],
                            'level': level,
                            'thinking': thinking,
                            'run': run,
                            'response': response_data['full_response'],
                            'thinking_content': response_data['thinking'],
                            'extracted_answer': extracted,
                            'correct': extracted == row['label'] if extracted else None,
                            'confidence': confidence,
                            'confidence_category': categorize_confidence(confidence),
                            'response_length': len(response_data['full_response'].split()),
                            'reasoning_markers': count_reasoning_markers(response_data['full_response']),
                            'uncertainty_markers': count_uncertainty_markers(response_data['full_response']),
                            'input_tokens': response_data['input_tokens'],
                            'output_tokens': response_data['output_tokens'],
                            'timestamp': datetime.now().isoformat(),
                        })
                    except Exception as e:
                        print(f"Error on {row['item_id']}: {e}")
                        ethics_results.append({
                            'item_id': row['item_id'],
                            'level': level,
                            'thinking': thinking,
                            'run': run,
                            'error': str(e),
                            'timestamp': datetime.now().isoformat(),
                        })
    
    # Save results
    ethics_results_df = pd.DataFrame(ethics_results)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    ethics_results_df.to_csv(f"{OUTPUT_DIR}/ethics_results_{timestamp}.csv", index=False)
    print(f"\nETHICS complete: {len(ethics_results_df)} observations")
    print(f"Saved to: {OUTPUT_DIR}/ethics_results_{timestamp}.csv")
else:
    print("Skipping ETHICS (RUN_ETHICS = False)")

## 6. Run MoralChoice Experiment

In [ ]:
if RUN_MORALCHOICE:
    print("=" * 60)
    print("RUNNING MORALCHOICE EXPERIMENT")
    print("=" * 60)
    
    mc_results = []
    mc_df = datasets['moralchoice']
    
    total_conditions = len(LEVELS) * len(THINKING_CONDITIONS) * N_RUNS
    condition_num = 0
    
    for run in range(N_RUNS):
        for thinking in THINKING_CONDITIONS:
            for level in LEVELS:
                condition_num += 1
                thinking_label = "ON" if thinking else "OFF"
                
                print(f"\n[{condition_num}/{total_conditions}] Run {run+1}, Level {level}, Thinking {thinking_label}")
                
                for _, row in tqdm(mc_df.iterrows(), total=len(mc_df),
                                   desc=f"L{level}-{thinking_label}"):
                    try:
                        response_data = run_single_moralchoice(row, level, thinking, INCLUDE_CONFIDENCE)
                        
                        extraction = extract_moralchoice_with_confidence(response_data['content'])
                        
                        mc_results.append({
                            'item_id': row['item_id'],
                            'context': row['context'][:100],
                            'option_a': row['option_a'][:100],
                            'option_b': row['option_b'][:100],
                            'ambiguity': row.get('ambiguity', None),
                            'level': level,
                            'thinking': thinking,
                            'run': run,
                            'response': response_data['full_response'],
                            'thinking_content': response_data['thinking'],
                            'extracted_answer': extraction['answer'],
                            'confidence': extraction['confidence'],
                            'confidence_category': extraction['confidence_category'],
                            'response_length': len(response_data['full_response'].split()),
                            'reasoning_markers': count_reasoning_markers(response_data['full_response']),
                            'uncertainty_markers': count_uncertainty_markers(response_data['full_response']),
                            'input_tokens': response_data['input_tokens'],
                            'output_tokens': response_data['output_tokens'],
                            'timestamp': datetime.now().isoformat(),
                        })
                    except Exception as e:
                        print(f"Error on {row['item_id']}: {e}")
                        mc_results.append({
                            'item_id': row['item_id'],
                            'level': level,
                            'thinking': thinking,
                            'run': run,
                            'error': str(e),
                            'timestamp': datetime.now().isoformat(),
                        })
    
    # Save results
    mc_results_df = pd.DataFrame(mc_results)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    mc_results_df.to_csv(f"{OUTPUT_DIR}/moralchoice_results_{timestamp}.csv", index=False)
    print(f"\nMoralChoice complete: {len(mc_results_df)} observations")
    print(f"Saved to: {OUTPUT_DIR}/moralchoice_results_{timestamp}.csv")
else:
    print("Skipping MoralChoice (RUN_MORALCHOICE = False)")

## 7. Run MORABLES Experiment

In [ ]:
if RUN_MORABLES:
    print("=" * 60)
    print("RUNNING MORABLES EXPERIMENT")
    print("=" * 60)
    
    morables_results = []
    morables_df = datasets['morables']
    
    total_conditions = len(LEVELS) * len(THINKING_CONDITIONS) * N_RUNS
    condition_num = 0
    
    for run in range(N_RUNS):
        for thinking in THINKING_CONDITIONS:
            for level in LEVELS:
                condition_num += 1
                thinking_label = "ON" if thinking else "OFF"
                
                print(f"\n[{condition_num}/{total_conditions}] Run {run+1}, Level {level}, Thinking {thinking_label}")
                
                for _, row in tqdm(morables_df.iterrows(), total=len(morables_df),
                                   desc=f"L{level}-{thinking_label}"):
                    try:
                        response_data = run_single_morables(row, level, thinking, INCLUDE_CONFIDENCE)
                        
                        extracted = extract_morables_answer(response_data['content'])
                        confidence = extract_confidence_score(response_data['content']) if INCLUDE_CONFIDENCE else None
                        
                        # Map letter to index for correctness check
                        letter_to_idx = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
                        extracted_idx = letter_to_idx.get(extracted)
                        correct = extracted_idx == row['correct_idx'] if extracted_idx is not None else None
                        
                        morables_results.append({
                            'item_id': row['item_id'],
                            'fable': row['fable'][:200],
                            'correct_idx': row['correct_idx'],
                            'correct_answer': ['A', 'B', 'C', 'D', 'E'][row['correct_idx']],
                            'level': level,
                            'thinking': thinking,
                            'run': run,
                            'response': response_data['full_response'],
                            'thinking_content': response_data['thinking'],
                            'extracted_answer': extracted,
                            'correct': correct,
                            'confidence': confidence,
                            'confidence_category': categorize_confidence(confidence),
                            'response_length': len(response_data['full_response'].split()),
                            'reasoning_markers': count_reasoning_markers(response_data['full_response']),
                            'uncertainty_markers': count_uncertainty_markers(response_data['full_response']),
                            'input_tokens': response_data['input_tokens'],
                            'output_tokens': response_data['output_tokens'],
                            'timestamp': datetime.now().isoformat(),
                        })
                    except Exception as e:
                        print(f"Error on {row['item_id']}: {e}")
                        morables_results.append({
                            'item_id': row['item_id'],
                            'level': level,
                            'thinking': thinking,
                            'run': run,
                            'error': str(e),
                            'timestamp': datetime.now().isoformat(),
                        })
    
    # Save results
    morables_results_df = pd.DataFrame(morables_results)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    morables_results_df.to_csv(f"{OUTPUT_DIR}/morables_results_{timestamp}.csv", index=False)
    print(f"\nMORABLES complete: {len(morables_results_df)} observations")
    print(f"Saved to: {OUTPUT_DIR}/morables_results_{timestamp}.csv")
else:
    print("Skipping MORABLES (RUN_MORABLES = False)")

## 8. Quick Results Summary

In [ ]:
def summarize_results(df, name):
    """Print quick summary of experiment results."""
    print(f"\n{'='*50}")
    print(f"{name} RESULTS SUMMARY")
    print(f"{'='*50}")
    
    # Filter to valid results
    valid = df[df['correct'].notna()]
    errors = df[df.get('error', pd.Series()).notna()] if 'error' in df.columns else pd.DataFrame()
    
    print(f"Total observations: {len(df)}")
    print(f"Valid extractions: {len(valid)} ({len(valid)/len(df)*100:.1f}%)")
    if len(errors) > 0:
        print(f"Errors: {len(errors)}")
    
    if len(valid) > 0:
        print(f"\nOverall accuracy: {valid['correct'].mean():.3f}")
        
        # By level
        print("\nAccuracy by level:")
        for level in sorted(valid['level'].unique()):
            level_data = valid[valid['level'] == level]
            print(f"  Level {level}: {level_data['correct'].mean():.3f} (n={len(level_data)})")
        
        # By thinking condition
        print("\nAccuracy by thinking condition:")
        for thinking in [False, True]:
            think_data = valid[valid['thinking'] == thinking]
            label = "ON" if thinking else "OFF"
            if len(think_data) > 0:
                print(f"  Thinking {label}: {think_data['correct'].mean():.3f} (n={len(think_data)})")

# Summarize each benchmark
if RUN_ETHICS and 'ethics_results_df' in dir():
    summarize_results(ethics_results_df, "ETHICS")

if RUN_MORALCHOICE and 'mc_results_df' in dir():
    # MoralChoice doesn't have "correct" - show extraction rate
    print(f"\n{'='*50}")
    print("MORALCHOICE RESULTS SUMMARY")
    print(f"{'='*50}")
    print(f"Total observations: {len(mc_results_df)}")
    valid = mc_results_df[mc_results_df['extracted_answer'].notna()]
    print(f"Valid extractions: {len(valid)} ({len(valid)/len(mc_results_df)*100:.1f}%)")
    print(f"\nAnswer distribution: {valid['extracted_answer'].value_counts().to_dict()}")
    if 'confidence' in valid.columns:
        print(f"Mean confidence: {valid['confidence'].mean():.1f}")

if RUN_MORABLES and 'morables_results_df' in dir():
    summarize_results(morables_results_df, "MORABLES")

## 9. Interactive Analysis (Optional)

Use this cell to explore your results interactively.

In [ ]:
# Example: View a specific response
# Uncomment and modify as needed

# if 'ethics_results_df' in dir():
#     # Show first response for Level 5 with thinking ON
#     sample = ethics_results_df[(ethics_results_df['level'] == 5) & (ethics_results_df['thinking'] == True)].iloc[0]
#     print(f"Item: {sample['item_id']}")
#     print(f"Scenario: {sample['scenario']}")
#     print(f"Correct: {sample['correct_answer']}")
#     print(f"Extracted: {sample['extracted_answer']}")
#     print(f"\nResponse:\n{sample['response']}")

In [ ]:
# Example: Pivot table of accuracy by level and thinking
# Uncomment and modify as needed

# if 'ethics_results_df' in dir():
#     valid = ethics_results_df[ethics_results_df['correct'].notna()]
#     pivot = valid.pivot_table(values='correct', index='level', columns='thinking', aggfunc='mean')
#     pivot.columns = ['Thinking OFF', 'Thinking ON']
#     display(pivot.round(3))

---

## Quick Reference: Parameter Options

| Parameter | Options | Description |
|-----------|---------|-------------|
| `RUN_ETHICS` | `True`/`False` | Run ETHICS benchmark |
| `RUN_MORALCHOICE` | `True`/`False` | Run MoralChoice benchmark |
| `RUN_MORABLES` | `True`/`False` | Run MORABLES benchmark |
| `LEVELS` | `[0,1,2,3,4,5]` | Which reflection levels to test |
| `THINKING_CONDITIONS` | `[False, True]` | OFF and/or ON |
| `SAMPLE_SIZE` | `int` or `None` | Items per benchmark (None = all) |
| `N_RUNS` | `int` | Number of runs for consistency |
| `INCLUDE_CONFIDENCE` | `True`/`False` | Ask for confidence scores |

### Level Descriptions
- **Level 0**: Direct answer only
- **Level 1**: Minimal instruction
- **Level 2**: Chain-of-Thought (step-by-step)
- **Level 3**: Structured analysis
- **Level 4**: Adversarial (consider counterarguments)
- **Level 5**: Two-Pass reflection (2 API calls per item)